# Agrupamento de dados de empreendimento por município e ano

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

pasta_dados = '../dados/'

df_empreend = pd.read_csv(pasta_dados + 'tratados/simu_carteira_empreendimentos.csv')
df_acidentes = pd.read_csv(pasta_dados + 'tratados/simu_acidentes_de_transportes.csv')
df_municipios = pd.read_csv(pasta_dados + 'simu_carteira_municipios.csv')

In [2]:
df_empreend.shape

(71248, 114)

In [3]:
for i in df_empreend.columns:
    print(i)

cod_mdr
cod_operacao
cod_saci
cod_convenio_siafi
cod_ag_operador
origem
dsc_concedente
dsc_fonte
dsc_unidade
cod_proposta
dsc_situacao_contrato_mdr
bln_carteira_mdr
bln_carteira_mdr_ativo
dsc_situacao_objeto_mdr
dsc_paralisada_mdr
bln_ativo
bln_ativo_boolean
cod_id_fonte
cod_id_pac
programa
acao
uf_SIGLA_UF
mun_MUNNOMEX
Código IBGE
empreendimento
vlr_repasse_financiamento
vlr_contrapartida
vlr_empenhado
vlr_desembolsado
vlr_desbloqueado
vlr_desembolsado_19
vlr_desembolsado_20
vlr_desembolsado_21
situacao_obra
situacao_obra_base_validacao
situacao_contrato
prc_fisico
dte_assinatura
dte_inicio_obra
dte_fim_obra
dte_ult_desembolso
dte_ult_desbloqueio
dte_ult_ref_bm
vlr_ult_desembolso
vlr_ult_repasse_desbloqueado
vlr_ult_contrapartida_desbloqueado
agente_financeiro
agente_financeiro_saci
dias_paralisados
situacao_obra_base_validacao_secex
situacao_obra_mdr
pop_beneficiada
emp_gerado
vlr_investimento
vlr_ult_investimento_desbloqueado
vlr_repasse_financiamento_num
vlr_contrapartida_num
vlr_i

## Agrupamento dos empreendimentos por conjunto município e ano

In [4]:
df_empreend[['vlr_investimento', 'pop_beneficiada']].isna().sum()

vlr_investimento    1036
pop_beneficiada        0
dtype: int64

In [5]:
# Não há colunas que possuem Código IBGE que não possuem vlr_investimento
df_empreend[df_empreend['vlr_investimento'].isnull() &df_empreend['Código IBGE'].notnull() ]

,cod_mdr,cod_operacao,cod_saci,cod_convenio_siafi,cod_ag_operador,origem,dsc_concedente,dsc_fonte,dsc_unidade,cod_proposta,dsc_situacao_contrato_mdr,bln_carteira_mdr,bln_carteira_mdr_ativo,dsc_situacao_objeto_mdr,dsc_paralisada_mdr,bln_ativo,bln_ativo_boolean,cod_id_fonte,cod_id_pac,programa,acao,uf_SIGLA_UF,mun_MUNNOMEX,Código IBGE,empreendimento,vlr_repasse_financiamento,vlr_contrapartida,vlr_empenhado,vlr_desembolsado,vlr_desbloqueado,vlr_desembolsado_19,vlr_desembolsado_20,vlr_desembolsado_21,situacao_obra,situacao_obra_base_validacao,situacao_contrato,prc_fisico,dte_assinatura,dte_inicio_obra,dte_fim_obra,dte_ult_desembolso,dte_ult_desbloqueio,dte_ult_ref_bm,vlr_ult_desembolso,vlr_ult_repasse_desbloqueado,vlr_ult_contrapartida_desbloqueado,agente_financeiro,agente_financeiro_saci,dias_paralisados,situacao_obra_base_validacao_secex,situacao_obra_mdr,pop_beneficiada,emp_gerado,vlr_investimento,vlr_ult_investimento_desbloqueado,vlr_repasse_financiamento_num,vlr_contrapartida_num,vlr_investimento_num,vlr_empenhado_num,vlr_desembolsado_num,vlr_desbloqueado_num,vlr_desembolsado_19_num,vlr_desembolsado_20_num,vlr_desembolsado_21_num,vlr_ult_desembolso_num,vlr_ult_repasse_desbloqueado_num,vlr_ult_contrapartida_desbloqueado_num,vlr_ult_investimento_desbloqueado_num,ano_assinatura,ano_inicio_obra,ano_fim_obra,ano_ult_desembolso,ano_ult_desbloqueio,ano_ult_ref_bm,ano,Populacao,Região,uf_REGIAO,Município,mun_coordenadas,mun_MUNNOME,mun_AMAZONIA,mun_FRONTEIRA,mun_CAPITAL,uf_CODIGO_UF,mun_LATITUDE,mun_LONGITUDE,mun_ALTITUDE,mun_AREA,mun_codigo_adotado,uf_NOME_UF,aux_emp_metro,aux_emp_pavimentacao,aux_emp_refrota,aux_emp_sinalizacao,aux_emp_calcamento,aux_emp_ciclovia,aux_emp_corredor,aux_emp_drenagem,aux_emp_ferrovia,aux_emp_rapido,aux_emp_vlt,aux_emp_recapeamento,aux_emp_aeroporto,aux_emp_revitalizacao,aux_emp_sarjeta,aux_emp_calcadas,aux_emp_praca,aux_emp_terminal,aux_emp_asfalto,aux_emp_ponte,aux_emp_parque,aux_emp_onibus,new_empreend


In [6]:
def agrupar_base_empreendimentos(df_empreend, ano=True):
    """
    Esta função agrupa um DataFrame de empreendimentos por município (e opcionalmente por ano),
    calculando várias estatísticas para cada grupo.

    Parâmetros:
    df_empreend (pandas.DataFrame): DataFrame de entrada que contém os dados dos empreendimentos.
    ano (bool, opcional): Se True, o DataFrame será agrupado por município e ano. Se False, será agrupado apenas por município.

    Retorna:
    pandas.DataFrame: DataFrame agrupado com novas colunas calculadas.

    """
    df = df_empreend.copy()

    aux_emp_columns = df.filter(like='aux_emp').columns

    agg_dict = {
        'vlr_investimento': 'sum',
        'Populacao': 'mean',
        'pop_beneficiada': 'sum',
        'Código IBGE': 'size'
    }

    # Para calcular o total de investimentos em cada categoria de empreendimentos, vamos, para cada coluna aux_emp, adicionar
    # uma operação de agregação ao dicionário e criar uma nova coluna no DataFrame que é igual ao valor de investimento se a coluna aux_emp não for nula, e zero caso contrário
    # Em seguida, quando agruparmos as colunas vamos somar o valor de investimento em cada categoria
    for column in aux_emp_columns:
        agg_dict[column] = lambda x: x.notnull().sum()

        new_column_name = column + '_vlr_investimento'
        
        # Calcula a soma dos valores da coluna "vlr_investimento" onde a coluna aux_emp correspondente não é nula
        df[new_column_name] = df.apply(lambda row: row['vlr_investimento'] if pd.notnull(row[column]) else 0, axis=1)
        
        agg_dict[new_column_name] = 'sum'

    # Cria colunas dummy para cada valor distinto na coluna 'programa' - Em seguida, faremos o mesmo que fizemos
    # com as categorias de empreendimentos para obter o valor do investimento em cada programa
    programa_dummies = pd.get_dummies(df['programa'], prefix='programa')

    df = pd.concat([df, programa_dummies], axis=1)

    for column in programa_dummies.columns:
        agg_dict[column] = 'sum'
        
        new_column_name = column + '_vlr_investimento'
        
        # Calcula a soma dos valores da coluna "vlr_investimento" onde a coluna 'programa' corresponde a cada valor distinto
        df[new_column_name] = df.apply(lambda row: row['vlr_investimento'] if row['programa'] == column.replace('programa_', '') else 0, axis=1)
        
        agg_dict[new_column_name] = 'sum'
    
    if ano:
        colunas_agrupar = ['Código IBGE', 'ano']
    else:
        colunas_agrupar = ['Código IBGE']

    df_agrupado_municipio_empreend = df.groupby(colunas_agrupar).agg(agg_dict)

    df_agrupado_municipio_empreend.rename(columns={'Código IBGE': 'num_total_empreendimentos' }, inplace=True)

    # Calcula o valor de investimento e a população beneficiada per capita
    df_agrupado_municipio_empreend['vlr_investimento_per_capita'] = df_agrupado_municipio_empreend['vlr_investimento'] / df_agrupado_municipio_empreend['Populacao']
    df_agrupado_municipio_empreend['pop_beneficiada_per_capita'] = df_agrupado_municipio_empreend['pop_beneficiada'] / df_agrupado_municipio_empreend['Populacao']

    df_agrupado_municipio_empreend = df_agrupado_municipio_empreend.reset_index()

    return df_agrupado_municipio_empreend

In [7]:
df_agrupado_municipio_empreend = agrupar_base_empreendimentos(df_empreend, ano=True)

In [8]:
df_agrupado_municipio_empreend.shape

(45979, 64)

In [9]:
df_agrupado_municipio_empreend.isna().sum()

Código IBGE                                  0
ano                                          0
vlr_investimento                             0
Populacao                                    0
pop_beneficiada                              0
                                            ..
programa_PRÓ COMUNIDADE_vlr_investimento     0
programa_SETOR PRIVADO                       0
programa_SETOR PRIVADO_vlr_investimento      0
vlr_investimento_per_capita                 13
pop_beneficiada_per_capita                  16
Length: 64, dtype: int64

## Combinação da base de empreendimentos com a de acidentes

In [10]:
df_acidentes

,Código IBGE,ano,Região,Pedestre_mortes,Ciclista_mortes,Motociclista_mortes,Ocup_triciclo_motor_mortes,Ocup_automovel_mortes,Ocup_caminhonete_mortes,Ocup_veic_transp_pesado_mortes,Ocup_onibus_mortes,Outros_mortes,total_mortes,taxa_mun_mortes,taxa_mun_pedestre_mortes,taxa_mun_ciclista_mortes,taxa_mun_motociclista_mortes,taxa_mun_automovel_mortes,taxa_mun_veiculo_pesado_mortes,taxa_mun_onibus_mortes,taxa_mun_outros_mortes,Pedestre_feridos,Ciclista_feridos,Motociclista_feridos,Ocup_triciclo_motor_feridos,Ocup_automovel_feridos,Ocup_caminhonete_feridos,Ocup_veic_transp_pesado_feridos,Ocup_onibus_feridos,Outros_feridos,total_feridos,taxa_mun_feridos,taxa_mun_pedestre_feridos,taxa_mun_ciclista_feridos,taxa_mun_motociclista_feridos,taxa_mun_automovel_feridos,taxa_mun_veiculo_pesado_feridos,taxa_mun_onibus_feridos,taxa_mun_outros_feridos,Populacao,taxa_mun_triciclo_mortes,taxa_mun_caminhonete_mortes,taxa_mun_triciclo_feridos,taxa_mun_caminhonete_feridos
0,1100015,2010,N,2,0,5,0,0,0,0,0,0,7.0,0.028698,0.008199,0.000000,0.020499,0.000000,0.000000,0.000000,0.000000,0,0,13,0,0,0,0,0,0,13.0,0.053296,0.000000,0.000000,0.053296,0.000000,0.000000,0.000000,0.000000,24392.0,0.0,0.000000,0.000000,0.000000
1,1100023,2010,N,0,1,2,0,0,0,0,1,29,33.0,0.036523,0.000000,0.001107,0.002214,0.000000,0.000000,0.001107,0.032096,0,1,37,0,1,1,0,0,3,43.0,0.047591,0.000000,0.001107,0.040950,0.001107,0.000000,0.000000,0.003320,90353.0,0.0,0.000000,0.000000,0.001107
2,1100031,2010,N,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6313.0,0.0,0.000000,0.000000,0.000000
3,1100049,2010,N,2,1,17,0,4,0,0,0,2,26.0,0.033090,0.002545,0.001273,0.021636,0.005091,0.000000,0.000000,0.002545,50,1,5,0,0,0,0,0,1,57.0,0.072543,0.063634,0.001273,0.006363,0.000000,0.000000,0.000000,0.001273,78574.0,0.0,0.000000,0.000000,0.000000
4,1100056,2010,N,3,0,1,0,0,0,0,0,0,4.0,0.023489,0.017617,0.000000,0.005872,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0,0,0,0,0,1.0,0.005872,0.000000,0.005872,0.000000,0.000000,0.000000,0.000000,0.000000,17029.0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55695,5222005,2019,CO,1,0,1,0,0,0,0,0,2,4.0,0.028854,0.007213,0.000000,0.007213,0.000000,0.000000,0.000000,0.014427,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13863.0,0.0,0.000000,0.000000,0.000000
55696,5222054,2019,CO,0,0,0,0,2,0,0,0,2,4.0,0.045751,0.000000,0.000000,0.000000,0.022875,0.000000,0.000000,0.022875,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8743.0,0.0,0.000000,0.000000,0.000000
55697,5222203,2019,CO,0,0,1,0,6,0,1,0,0,8.0,0.129639,0.000000,0.000000,0.016205,0.097229,0.016205,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6171.0,0.0,0.000000,0.000000,0.000000
55698,5222302,2019,CO,0,0,0,0,1,0,0,0,2,3.0,0.051538,0.000000,0.000000,0.000000,0.017179,0.000000,0.000000,0.034358,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5821.0,0.0,0.000000,0.000000,0.000000


Sabemos que df_acidentes é único por conjunto Código IBGE e ano (isso foi verificado no notebook tratamento_acidentes). Assim, podemos fazer o merge da base agrupada de empreendimentos com a base de acidentes, para obter em um único dataframe dados de empreendimentos e de acidentes por conjunto município e ano.

In [11]:
df_agrupado_municipio_empreend.shape

(45979, 64)

In [12]:
df_acidentes.shape

(55700, 44)

In [13]:
list(set(df_agrupado_municipio_empreend.columns).intersection(set(df_acidentes.columns)))

['Código IBGE', 'ano', 'Populacao']

In [14]:
df_acidentes = df_acidentes.drop('Populacao', axis = 1)

In [15]:
def merge_dataframes_empreend_acidentes(df_agrupado_municipio_empreend, df_acidentes, merge_on = ['Código IBGE', 'ano']):
    """
    Função que realiza a junção de dois dataframes, um contendo dados de empreendimentos e outro contendo dados de acidentes.
    
    Parâmetros:
    df_agrupado_municipio_empreend (pandas.DataFrame): DataFrame contendo dados de empreendimentos agrupados por município.
    df_acidentes (pandas.DataFrame): DataFrame contendo dados de acidentes.
    merge_on (list, opcional): Lista de colunas para realizar a junção. Por padrão é ['Código IBGE', 'ano'].
    
    Retorna:
    df_merge_empreendimentos_acidentes (pandas.DataFrame): DataFrame resultante da junção 'outer'.
    df_merge_empreendimentos_acidentes_sem_nulo (pandas.DataFrame): DataFrame resultante da junção 'inner', contendo apenas os conjuntos município e ano
                                                                    que contém tanto algum valor de empreendimento quanto o número de acidentes.
    """
    df_merge_empreendimentos_acidentes = pd.merge(df_agrupado_municipio_empreend, df_acidentes, on=merge_on, how= 'outer')
    
    # Para evitar que o modelo seja treinado com muitos dados nulos, vamos também fazer um inner join para selecionar
    # apenas os conjuntos município e ano que contém tanto algum valor de empreendimento quanto o número de acidentes
    df_merge_empreendimentos_acidentes_sem_nulo = pd.merge(df_agrupado_municipio_empreend, df_acidentes, on=merge_on, how='inner')
    
    return df_merge_empreendimentos_acidentes, df_merge_empreendimentos_acidentes_sem_nulo


In [16]:
df_merge_empreendimentos_acidentes, df_merge_empreendimentos_acidentes_sem_nulo = merge_dataframes_empreend_acidentes(df_agrupado_municipio_empreend, df_acidentes)

In [17]:
unique_rows = df_merge_empreendimentos_acidentes.duplicated(subset=['Código IBGE', 'ano'], keep=False)

if unique_rows.sum() == 0:
    print("O conjunto dos valores das colunas 'Código IBGE' e 'ano' é único.")
else:
    print("O conjunto dos valores das colunas 'Código IBGE' e 'ano' não é único.")

O conjunto dos valores das colunas 'Código IBGE' e 'ano' é único.


In [18]:
df_merge_empreendimentos_acidentes.isna().sum().sum()

3206468

In [19]:
df_merge_empreendimentos_acidentes_sem_nulo.isna().sum().sum()

0

In [20]:
df_merge_empreendimentos_acidentes.to_csv('../dados/tratados/agrupamento_empreend_acidentes_por_municipio_e_ano.csv', index=False)

df_merge_empreendimentos_acidentes_sem_nulo.to_csv('../dados/tratados/agrupamento_empreend_acidentes_por_municipio_e_ano_sem_nulos.csv', index=False)

# Agrupar tudo do município, juntando os anos

In [21]:
df_agrupado_municipio_empreend_anos_juntos = agrupar_base_empreendimentos(df_empreend, ano=False)

In [22]:
df_agrupado_municipio_empreend_anos_juntos

,Código IBGE,vlr_investimento,Populacao,pop_beneficiada,num_total_empreendimentos,aux_emp_metro,aux_emp_metro_vlr_investimento,aux_emp_pavimentacao,aux_emp_pavimentacao_vlr_investimento,aux_emp_refrota,aux_emp_refrota_vlr_investimento,aux_emp_sinalizacao,aux_emp_sinalizacao_vlr_investimento,aux_emp_calcamento,aux_emp_calcamento_vlr_investimento,aux_emp_ciclovia,aux_emp_ciclovia_vlr_investimento,aux_emp_corredor,aux_emp_corredor_vlr_investimento,aux_emp_drenagem,aux_emp_drenagem_vlr_investimento,aux_emp_ferrovia,aux_emp_ferrovia_vlr_investimento,aux_emp_rapido,aux_emp_rapido_vlr_investimento,aux_emp_vlt,aux_emp_vlt_vlr_investimento,aux_emp_recapeamento,aux_emp_recapeamento_vlr_investimento,aux_emp_aeroporto,aux_emp_aeroporto_vlr_investimento,aux_emp_revitalizacao,aux_emp_revitalizacao_vlr_investimento,aux_emp_sarjeta,aux_emp_sarjeta_vlr_investimento,aux_emp_calcadas,aux_emp_calcadas_vlr_investimento,aux_emp_praca,aux_emp_praca_vlr_investimento,aux_emp_terminal,aux_emp_terminal_vlr_investimento,aux_emp_asfalto,aux_emp_asfalto_vlr_investimento,aux_emp_ponte,aux_emp_ponte_vlr_investimento,aux_emp_parque,aux_emp_parque_vlr_investimento,aux_emp_onibus,aux_emp_onibus_vlr_investimento,programa_AVANÇAR PÚBLICO,programa_AVANÇAR PÚBLICO_vlr_investimento,programa_EMENDAS,programa_EMENDAS_vlr_investimento,programa_PAC FIN,programa_PAC FIN_vlr_investimento,programa_PAC OGU,programa_PAC OGU_vlr_investimento,programa_PRÓ COMUNIDADE,programa_PRÓ COMUNIDADE_vlr_investimento,programa_SETOR PRIVADO,programa_SETOR PRIVADO_vlr_investimento,vlr_investimento_per_capita,pop_beneficiada_per_capita
0,1100015.0,2.530000e+06,2.365733e+04,3034.0,3,0,0.000000e+00,2,1.530000e+06,0,0.0,0,0.00,0,0.0,0,0.00,0,0.000000e+00,0,0.000000e+00,0,0.0,0,0.000000e+00,0,0.000000e+00,0,0.00,0,0.000000e+00,0,0.0,0,0.00,0,0.00,0,0.00,0,0.000000e+00,0,0.00,0,0.00,0,0.00,0,0.00,0,0.0,3,2.530000e+06,0,0.000000e+00,0,0.000000e+00,0,0.0,0,0.0,106.943583,0.128248
1,1100023.0,5.172818e+07,8.014420e+04,60220.0,15,0,0.000000e+00,10,4.591718e+07,0,0.0,0,0.00,0,0.0,1,1251042.30,0,0.000000e+00,6,7.144334e+06,0,0.0,0,0.000000e+00,0,0.000000e+00,0,0.00,0,0.000000e+00,0,0.0,0,0.00,2,914793.15,0,0.00,0,0.000000e+00,6,3092469.97,0,0.00,0,0.00,0,0.00,0,0.0,14,1.488608e+07,1,3.684211e+07,0,0.000000e+00,0,0.0,0,0.0,645.438850,0.751396
2,1100031.0,3.497805e+05,7.242000e+03,410.0,2,0,0.000000e+00,2,3.497805e+05,0,0.0,0,0.00,0,0.0,0,0.00,0,0.000000e+00,1,2.658500e+05,0,0.0,0,0.000000e+00,0,0.000000e+00,0,0.00,0,0.000000e+00,0,0.0,0,0.00,0,0.00,0,0.00,0,0.000000e+00,1,83930.51,0,0.00,0,0.00,0,0.00,0,0.0,2,3.497805e+05,0,0.000000e+00,0,0.000000e+00,0,0.0,0,0.0,48.298883,0.056614
3,1100049.0,8.067858e+07,7.571084e+04,88575.0,19,1,2.294597e+06,14,6.605357e+07,0,0.0,0,0.00,0,0.0,0,0.00,0,0.000000e+00,4,2.278389e+07,0,0.0,0,0.000000e+00,0,0.000000e+00,1,530600.00,0,0.000000e+00,0,0.0,0,0.00,0,0.00,0,0.00,0,0.000000e+00,7,11031705.32,0,0.00,0,0.00,0,0.00,0,0.0,17,3.479553e+07,2,4.588305e+07,0,0.000000e+00,0,0.0,0,0.0,1065.614652,1.169912
4,1100056.0,9.547742e+06,1.704967e+04,10741.0,9,0,0.000000e+00,7,8.965754e+06,0,0.0,0,0.00,0,0.0,0,0.00,0,0.000000e+00,3,3.787707e+06,0,0.0,0,0.000000e+00,0,0.000000e+00,0,0.00,0,0.000000e+00,0,0.0,2,1380880.77,0,0.00,0,0.00,0,0.000000e+00,1,513626.74,0,0.00,0,0.00,0,0.00,0,0.0,9,9.547742e+06,0,0.000000e+00,0,0.000000e+00,0,0.0,0,0.0,559.995844,0.629983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5540,5222005.0,4.378918e+06,1.335321e+04,5352.0,14,0,0.000000e+00,10,3.531364e+06,0,0.0,0,0.00,0,0.0,1,102100.00,0,0.000000e+00,0,0.000000e+00,0,0.0,0,0.000000e+00,0,0.000000e+00,1,253453.61,0,0.000000e+00,0,0.0,0,0.00,1,102100.00,0,0.00,0,0.000000e+00,0,0.00,0,0.00,0,0.00,0,0.00,0,0.0,14,4.378918e+06,0,0.000000e+00,0,0.000000e+00,0,0.0,0,0.0,327.929889,0.400802
5541,522

In [23]:
df_agrupado_municipio_empreend_anos_juntos.shape

(5545, 63)

In [24]:
unique_rows = df_agrupado_municipio_empreend_anos_juntos.duplicated(subset=['Código IBGE'], keep=False)

if unique_rows.sum() == 0:
    print("O conjunto dos valores da coluna 'Código IBGE' é único.")
else:
    print("O conjunto dos valores da coluna 'Código IBGE' não é único.")


O conjunto dos valores da coluna 'Código IBGE' é único.


In [25]:
df_acidentes_agrupado_ano = df_acidentes.groupby('Código IBGE').mean()
df_acidentes_agrupado_ano.shape

(5570, 41)

In [26]:
df_merge_empreendimentos_acidentes_anos_juntos, df_merge_empreendimentos_acidentes_anos_juntos_sem_nulo = merge_dataframes_empreend_acidentes(df_agrupado_municipio_empreend_anos_juntos,
                                                                                                                                  df_acidentes_agrupado_ano, merge_on = ['Código IBGE'])

In [27]:
unique_rows = df_merge_empreendimentos_acidentes_anos_juntos.duplicated(subset=['Código IBGE'], keep=False)

if unique_rows.sum() == 0:
    print("O conjunto dos valores da coluna 'Código IBGE' é único.")
else:
    print("O conjunto dos valores da coluna 'Código IBGE' não é único.")


O conjunto dos valores da coluna 'Código IBGE' é único.


In [28]:
df_merge_empreendimentos_acidentes_anos_juntos.isna().sum()

Código IBGE                      0
vlr_investimento                25
Populacao                       25
pop_beneficiada                 25
num_total_empreendimentos       25
                                ..
taxa_mun_outros_feridos          0
taxa_mun_triciclo_mortes         0
taxa_mun_caminhonete_mortes      0
taxa_mun_triciclo_feridos        0
taxa_mun_caminhonete_feridos     0
Length: 104, dtype: int64

In [29]:
df_merge_empreendimentos_acidentes_anos_juntos_sem_nulo.isna().sum()

Código IBGE                     0
vlr_investimento                0
Populacao                       0
pop_beneficiada                 0
num_total_empreendimentos       0
                               ..
taxa_mun_outros_feridos         0
taxa_mun_triciclo_mortes        0
taxa_mun_caminhonete_mortes     0
taxa_mun_triciclo_feridos       0
taxa_mun_caminhonete_feridos    0
Length: 104, dtype: int64

In [30]:
df_merge_empreendimentos_acidentes_anos_juntos.to_csv('../dados/tratados/agrupamento_empreend_acidentes_por_municipio.csv', index=False)

df_merge_empreendimentos_acidentes_anos_juntos_sem_nulo.to_csv('../dados/tratados/agrupamento_empreend_acidentes_por_municipio_sem_nulos.csv', index=False)